# TalkingData AdTracking Fraud Detection 
This dataset is huge , one of the biggest i have ever seen.

For this dataset, the objective is to predict whether a user will download an app after clicking a mobile app advertisement. Since, they have given a small training sample, we will first explore the dataset using this sample training set.

First let us import the necessary libraries and have a look at the data



In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

train=pd.read_csv('mnt_sample/ssd/kaggle-talkingdata2/competition_files/train_sample.csv')
train.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0


In [6]:
train.shape

(100000, 8)

## Data fields

Each row of the training data contains a click record, with the following features.

- ip: ip address of click.
- app: app id for marketing.
- device: device type id of user mobile phone (e.g., iphone 6 plus, iphone 7, huawei mate 7, etc.)
- os: os version id of user mobile phone
- channel: channel id of mobile ad publisher
- click_time: timestamp of click (UTC)
- attributed_time: if user download the app for after clicking an ad, this is the time of the app download
- is_attributed: the target that is to be predicted, indicating the app was downloaded
Note that ip, app, device, os, and channel are encoded.

The test data is similar, with the following differences:

- click_id: reference for making predictions
- is_attributed: not included


The evaluation metric used is ROC curve, the AUC of the ROC graph. Let us have a look at the actual train and test set below 

In [9]:
train=pd.read_csv('mnt/ssd/kaggle-talkingdata2/competition_files/train.csv',nrows=1000000)
test=pd.read_csv('test.csv',nrows=1000000)

In [10]:
train.shape,test.shape

((1000000, 8), (1000000, 7))

In [13]:
#test file looks as suspected
test.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00
